In [252]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [15]:
df = pd.read_csv("data/datathon_data.csv")
categorical_cols = ["WAERS", "BUKRS", "KTOSL", "PRCTR", "BSCHL", "HKONT"]


# Get rare values
How do they relate to anomalies

In [75]:
import pandas as pd

def compute_anomaly_ratios(df, categorical_cols = categorical_cols, label_col='label', anomaly_value='anomal'):
    info = {}
    
    for col in categorical_cols:
        grouped = df.groupby(col)
        ratio = grouped[label_col].value_counts(normalize=True).unstack().fillna(0).get(anomaly_value, 0)
        info[col] = pd.DataFrame({"ratio": ratio, "n_unique": df.groupby(col).size()}).reset_index()
        info[col] = info[col].rename(columns={col: "value"})
        info[col]["variable"] = col
        
    anomyly_ratios = pd.concat(info.values(), axis=0, ignore_index=True)
    return anomyly_ratios

# Example usage
categorical_columns = ["WAERS", "BUKRS", "KTOSL", "PRCTR", "BSCHL", "HKONT"]
anomyly_ratios = compute_anomaly_ratios(df, categorical_columns)


In [18]:
all_anomaly_1val = anomyly_ratios[anomyly_ratios.ratio == 1]
all_anomaly_1val.variable.unique()
all_anomaly_1val[all_anomaly_1val.n_unique > 1]

#anomyly_ratios[(anomyly_ratios.ratio > 0.5) & (anomyly_ratios.ratio < 1)]
rare_values = anomyly_ratios[anomyly_ratios.n_unique < 10]


all rare values are in anomaly transtaction

how many anomalies are there that don't contain rare values?
only 30

In [256]:
df_anomal = df[df.label == "anomal"]
df_no_rare = df.copy()
for _,r in rare_values.iterrows():
    df_no_rare = df_no_rare[df_no_rare.get(r.variable) != r.value]

df_no_rare_anomal = df_anomal.copy()
for _,r in rare_values.iterrows():
    df_no_rare_anomal = df_no_rare_anomal[df_no_rare_anomal.get(r.variable) != r.value]

In [26]:
compute_anomaly_ratios(df_no_rare_anomal, categorical_cols).sort_values(by = "n_unique", ascending= False)

,value,ratio,n_unique,variable
0,C1,1.0,30,WAERS
4,C1,1.0,30,KTOSL
35,A1,1.0,30,BSCHL
36,B1,1.0,30,HKONT
1,C11,1.0,28,BUKRS
3,C20,1.0,1,BUKRS
2,C17,1.0,1,BUKRS
5,C18,1.0,1,PRCTR
6,C20,1.0,1,PRCTR
9,C26,1.0,1,PRCTR


## Are BURKS (Currency) and Currency (WAERS) related like they should? 
all that aren't are anomalies

In [ ]:
df.loc[df["BUKRS"].str.slice(0,2) != df["WAERS"]].groupby("label").count()

,BELNR,WAERS,BUKRS,KTOSL,PRCTR,BSCHL,HKONT,DMBTR,WRBTR
label,,,,,,,,,
anomal,71,71,71,71,71,71,71,71,71


In [71]:
df_burks_waers_match = df_no_rare.loc[df_no_rare["BUKRS"].str.slice(0,2) == df_no_rare["WAERS"]]
df_burks_waers_match.shape

(29, 10)

In [78]:
compute_anomaly_ratios(df_burks_waers_match).sort_values(by = "n_unique", ascending= False)

,value,ratio,n_unique,variable
0,C1,1.0,29,WAERS
34,B1,1.0,29,HKONT
3,C1,1.0,29,KTOSL
33,A1,1.0,29,BSCHL
1,C11,1.0,28,BUKRS
2,C17,1.0,1,BUKRS
4,C20,1.0,1,PRCTR
7,C26,1.0,1,PRCTR
8,C30,1.0,1,PRCTR
5,C21,1.0,1,PRCTR


## Relations of Numerical values to anomaly

In [83]:
df_burks_waers_match

,BELNR,WAERS,BUKRS,KTOSL,PRCTR,BSCHL,HKONT,DMBTR,WRBTR,label
32317,32317,C1,C11,C1,C53,A1,B1,910653.005238,54439.211421,anomal
34058,34058,C1,C11,C1,C91,A1,B1,910668.887465,54440.856583,anomal
34059,34059,C1,C11,C1,C64,A1,B1,910631.632279,54443.283794,anomal
43399,43399,C1,C11,C1,C79,A1,B1,910650.440131,54435.055247,anomal
97954,97954,C1,C11,C1,C68,A1,B1,910648.636111,54442.618111,anomal
144286,144286,C1,C11,C1,C76,A1,B1,910645.076191,54448.440199,anomal
170320,170320,C1,C11,C1,C30,A1,B1,910645.965114,54452.809780,anomal
183064,183064,C1,C11,C1,C82,A1,B1,910648.873494,54436.705575,anomal
213105,213105,C1,C17,C1,C20,A1,B1,910664.385738,54436.897572,anomal
220927,220927,C1,C11,C1,C60,A1,B1,910648.545092,54446.382830,anomal


In [ ]:
index_high = df[df['label'] == 'regular'][['DMBTR', 'WRBTR']].prod(axis=1).sort_values(ascending = False)[1:20].index
df.iloc[index_high]
#df_burks_waers_match[['DMBTR', 'WRBTR']].prod(axis=1).sort_values()

,BELNR,WAERS,BUKRS,KTOSL,PRCTR,BSCHL,HKONT,DMBTR,WRBTR,label,mul
236394,236394,C7,C72,C1,C13,A3,B1,10000000.0,57316255.05,regular,1.000000e+14
71811,71811,C5,C59,C1,C19,A3,B1,10000000.0,50399045.08,regular,1.000000e+14
449918,449918,C1,C15,C1,C10,A1,B1,10000000.0,49585040.37,regular,1.000000e+14
483430,483430,C5,C50,C2,C27,A3,B3,10000000.0,47316255.05,regular,1.000000e+14
275309,275309,C7,C78,C4,C49,A3,B3,10000000.0,45674547.89,regular,1.000000e+14
36767,36767,C5,C57,C6,C67,A3,B2,10000000.0,44892193.09,regular,1.000000e+14
193465,193465,C7,C79,C2,C26,A3,B3,10000000.0,42542664.27,regular,1.000000e+14
383728,383728,C1,C13,C9,C97,A1,B1,10000000.0,40399045.08,regular,1.000000e+14
100769,100769,C1,C16,C1,C19,A1,B1,10000000.0,39585040.37,regular,1.000000e+14
207617,207617,C3,C31,C2,C20,A3,B3,10000000.0,37670038.08,regular,1.000000e+14


no direct relationship

there are some variables which have only 1 unique value. Look at distribution with those

In [187]:
one_unique_cols = df_burks_waers_match.columns[df_burks_waers_match.nunique() == 1]
one_unique_cols = one_unique_cols.drop("label")

# Step 2: Get the unique value of each of those columns
unique_values = df_burks_waers_match[one_unique_cols].iloc[0]

In [234]:
df_like_unique_anomal = df.copy()

for id, val in unique_values.items():
    df_like_unique_anomal = df_like_unique_anomal[df_like_unique_anomal.get(id) == val]

In [235]:
df_like_unique_anomal[df_like_unique_anomal['label'] == 'regular'][['DMBTR', 'WRBTR']].prod(axis=1).sort_values(ascending = False)

#df_burks_waers_match[['DMBTR', 'WRBTR']].prod(axis=1).max

449918    4.958504e+14
100769    3.958504e+14
32986     3.489219e+14
468135    3.254266e+14
437041    2.958504e+14
              ...     
532878    0.000000e+00
532873    0.000000e+00
532859    0.000000e+00
532856    0.000000e+00
108       0.000000e+00
Length: 141894, dtype: float64

is there anything just once?

In [257]:
unique_counts = {col: df_no_rare[col].nunique() for col in categorical_cols}
print("Unique value counts per categorical column:")
print(unique_counts)
np.array([i for i in unique_counts.values()]).prod()

Unique value counts per categorical column:
{'WAERS': 9, 'BUKRS': 90, 'KTOSL': 9, 'PRCTR': 90, 'BSCHL': 3, 'HKONT': 3}


np.int64(5904900)